In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load pre-trained FER model
fer_model = load_model("ferplus_model.h5")

# Optionally freeze early layers
for layer in fer_model.layers[:-3]:  # Freeze all but last 3 layers
    layer.trainable = False

In [9]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model

# Extract existing feature layers
features = fer_model.layers[-2].output  # Get last feature extraction layer

# Add demographic prediction branch
age_gender_output = Dense(2, activation="softmax", name="age_gender")(features)

# New model with both emotion and demographic branches
model = Model(inputs=fer_model.input, outputs=[fer_model.output, age_gender_output])


In [10]:
alpha = 0.5  # Adjust this to balance fairness and accuracy

# Define losses
emotion_loss = tf.keras.losses.SparseCategoricalCrossentropy()
demographic_loss = tf.keras.losses.SparseCategoricalCrossentropy()

def custom_loss(y_true, y_pred):
    y_true_emotion, y_true_demo = y_true
    y_pred_emotion, y_pred_demo = y_pred

    # Compute losses
    loss_emo = emotion_loss(y_true_emotion, y_pred_emotion)
    loss_demo = demographic_loss(y_true_demo, y_pred_demo)

    # Apply fairness adjustment
    total_loss = loss_emo - alpha * loss_demo
    return total_loss


In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np

def load_dataset(csv_path):
    df = pd.read_csv(csv_path)

    # Assuming CSV has: ['image_path', 'emotion_label', 'age_gender_label']
    image_paths = df["image"].values
    emotion_labels = df["label"].values
    demographic_labels = df["Gender"].values  # Age/Gender categories
    
    images = np.array([tf.keras.preprocessing.image.img_to_array(
                        tf.keras.preprocessing.image.load_img(path, color_mode="grayscale", target_size=(48,48))
                      ) for path in image_paths])

    # Normalize images
    images = images / 255.0

    return images, emotion_labels, demographic_labels

# Load dataset (example structure)
emotion_labels, demographic_labels = load_dataset("test_labels.csv") 

# Convert to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(((images, labels), (emotion_labels, demographic_labels)))
train_dataset = train_dataset.batch(32).shuffle(1000)

FileNotFoundError: [Errno 2] No such file or directory: 'test_0001_aligned.jpg'

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_loss)

# Train model
model.fit(train_dataset, epochs=10)

In [ ]:
# Evaluate on validation/test set
test_loss, test_emotion_acc, test_demo_acc = model.evaluate(test_dataset)

print(f"Emotion Accuracy: {test_emotion_acc:.4f}")
print(f"Demographic Fairness Accuracy: {test_demo_acc:.4f}")